# Understand the behavior of number of the shots and simulator

Since the number of the shots from the simulator play role in the accuracy and the expressibilty, we want to further study the effect of the those parameters. 

In this document, we will only focus on the arbitary circuit since it should give identical result with the harr distribution.

In [1]:
from matplotlib import pyplot
from qiskit import *
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import IBMQ, Aer, execute,assemble,QuantumCircuit, aqua
from qiskit.visualization import plot_histogram, plot_bloch_vector, plot_bloch_multivector
from qiskit.quantum_info import Statevector
from qiskit.extensions import *
provider = IBMQ.load_account()

from qiskit.quantum_info import random_unitary
from qiskit.providers.aer import QasmSimulator, StatevectorSimulator, UnitarySimulator
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import math
from math import pi, sqrt

from scipy.special import rel_entr
from random import seed
from random import random
import cmath

C:\Users\Saesun Kim\anaconda3\envs\name_of_my_env\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:

#Possible Bin
bins_list=[];
for i in range(76):
    bins_list.append((i)/75)
#Center of the Bean
bins_x=[]    
for i in range(75):
    bins_x.append(bins_list[1]+bins_list[i])
def P_harr(l,u,N):
    return (1-l)**(N-1)-(1-u)**(N-1)
#Harr historgram
P_harr_hist=[]
for i in range(75):
    P_harr_hist.append(P_harr(bins_list[i],bins_list[i+1],2))    
#Imaginary    
j=(-1)**(1/2)

In [3]:
from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()

ibmqfactory.load_account:WARNING:2021-03-21 00:50:30,964: Credentials are already in use. The existing account in the session will be replaced.


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') fr

In [5]:
backend = 'ibmq_valencia'# you may choice here a different backend

In [ ]:
fidelity_dic = {'ibmq_athens': 0.925110, 'ibmq_valencia': 0.809101, 'ibmq_ourense': 0.802380,
                "ibmqx2": 0.627392, 'ibmq_santiago': 0.919399, 'ibmq_vigo': 0.908840, 'ideal_device': 1.0}
QV_dic = {'ibmq_athens': 32.0, 'ibmq_valencia': 16.0, 'ibmq_ourense': 8.0, 
          "ibmqx2": 8.0, 'ibmq_santiago': 32.0, 'ibmq_vigo': 16.0, 'ideal_device': np.inf}
dev_dic =  {'ibmq_santiago': "San",'ibmq_athens': "Ath", 'ibmq_valencia': "Val", 'ibmq_vigo': 'Vig','ibmq_ourense': "Our",
               "ibmqx2": 'Yor',  'ideal_device': "Ide"}

## Arbitary Unitary Qasm_simulator

In [8]:
backend = provider.get_backend('ibmq_vigo')
for x in range(1,20,1):
    print()
arr = []
for nsh in range(1,20,1):
    arr.append([])
    for lp in range(1,10,1):
        nshot=int(round(1000*(nsh)**1.5,0))
        nparam=1000*lp
        fidelity=[]    
        for x in range(nparam):
            qr = QuantumRegister(1)
            cr = ClassicalRegister(1)
            qc = QuantumCircuit(qr, cr)

            u13=UnitaryGate(random_unitary(2))
            qc.append(u13, [qr[0]] )
            u13=UnitaryGate(random_unitary(2))
            qc.append(u13, [qr[0]] )

            qc.measure(qr[0],cr[0])
            job = execute(qc, backend, shots=nshot)
            result = job.result()
            count =result.get_counts()

            if '0' in count and '1' in count:
                ratio=count['0']/nshot
            elif '0' in count and '1' not in count:
                ratio=count['0']/nshot
            else:
                ratio=0

            fidelity.append(ratio)   
            #Kullback Leibler divergence
        weights = np.ones_like(fidelity)/float(len(fidelity))            
        P_U_hist=np.histogram(fidelity, bins=bins_list, weights=weights, range=[0, 1])[0];
        #Kullback Leibler divergence
        print(nshot,'shots per simulation',nparam,'distribution size')
        arr[nsh-1].append(sum(rel_entr(P_U_hist, P_harr_hist)))


QiskitBackendNotFoundError: 'No backend matches the criteria'

In [ ]:
import sys
import numpy
qasm=arr
numpy.set_printoptions(threshold=sys.maxsize)
np.savetxt("KLDivg_qasm.txt",qasm,fmt='%.2f')

## Arbitary StatevectorSimulator

In [ ]:
backend = Aer.get_backend('statevector_simulator')
for x in range(1,20,1):
    print()
arr = []
for nsh in range(1,20,1):
    arr.append([])
    for lp in range(1,10,1):
        nshot=int(round(1000*(nsh)**1.5,0))
        nparam=1000*lp
        fidelity=[]    
        for x in range(nparam):
            qr = QuantumRegister(1)
            cr = ClassicalRegister(1)
            qc = QuantumCircuit(qr, cr)

            u13=UnitaryGate(random_unitary(2))
            qc.append(u13, [qr[0]] )
            u13=UnitaryGate(random_unitary(2))
            qc.append(u13, [qr[0]] )

            qc.measure(qr[0],cr[0])
            job = execute(qc, backend, shots=nshot)
            result = job.result()
            count =result.get_counts()

            if '0' in count and '1' in count:
                ratio=count['0']/nshot
            elif '0' in count and '1' not in count:
                ratio=count['0']/nshot
            else:
                ratio=0

            fidelity.append(ratio)   
            #Kullback Leibler divergence
        weights = np.ones_like(fidelity)/float(len(fidelity))            
        P_U_hist=np.histogram(fidelity, bins=bins_list, weights=weights, range=[0, 1])[0];
        #Kullback Leibler divergence
        print(nshot,'shots per simulation',nparam,'distribution size')
        arr[nsh-1].append(sum(rel_entr(P_U_hist, P_harr_hist)))


In [ ]:
import sys
import numpy
statevector=arr
numpy.set_printoptions(threshold=sys.maxsize)
np.savetxt("KLDivg_statevector.txt",statevector,fmt='%.2f')

In [ ]:
##Plot

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

#loading dataset

x = [i+1 for i in range(19)]
y = [i[0] for i in arr]
plt.plot(x, y, 'o', color='red', label='L=1');
x = [i+1 for i in range(19)]
y = [i[1] for i in arr]
plt.plot(x, y, 'o', color='blue', label='L=2');
x = [i+1 for i in range(19)]
y = [i[2] for i in arr]
plt.plot(x, y, 'o', color='black', label='L=3');
x = [i+1 for i in range(19)]
y = [i[3] for i in arr]
plt.plot(x, y, 'o', color='green', label='L=4');
x = [i+1 for i in range(19)]
y = [i[3] for i in arr]
plt.plot(x, y, 'o', color='purple', label='L=5');
plt.legend(loc='upper right')

plt.yscale('log',base=10) 
plt.xlabel('Circuit ID')
plt.ylabel('Expressibility')

# Create names on the x axis
plt.xticks([i+1 for i in range(19)])
plt.show()